In [42]:
# Time Series Forecasting para Previsão de Vendas
# Projeto completo com múltiplas abordagens de Machine Learning

# Instalação das bibliotecas necessárias
!pip install -q yfinance plotly prophet xgboost lightgbm catboost scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Time Series Forecasting para Previsão de Vendas
# Projeto completo com múltiplas abordagens de Machine Learning

In [43]:
# Instalação das bibliotecas necessárias
!pip install -q yfinance plotly prophet xgboost lightgbm catboost scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

### Geração de Dados de Vendas (Simulados)

In [44]:
# Geração de Dados de Vendas (Simulados)
# Pode ser substituído pela carga de um dataset real

np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
sales = (np.random.randint(1000, 2000, size=len(dates)) +
         np.sin(np.arange(len(dates))/365*2*np.pi)*200 +
         np.cos(np.arange(len(dates))/7*2*np.pi)*50 +
         np.random.randn(len(dates))*50).astype(int)
sales[sales < 0] = 0

df = pd.DataFrame({'date': dates, 'sales': sales})
df.set_index('date', inplace=True)
display(df.head())

,sales
date,
2020-01-01,1164
2020-01-02,1393
2020-01-03,1762
2020-01-04,1197
2020-01-05,1120


### Análise Exploratória de Dados (AED)

In [45]:
# Visualização dos dados
fig = go.Figure(data=[go.Scatter(x=df.index, y=df['sales'], mode='lines')])
fig.update_layout(title='Série Temporal de Vendas Diárias',
                  xaxis_title='Data',
                  yaxis_title='Vendas',
                  template='plotly_white')
fig.show()

# Análise de sazonalidade (média mensal e semanal)
monthly_sales = df['sales'].groupby(df.index.month).mean()
weekday_sales = df['sales'].groupby(df.index.weekday).mean()

fig = make_subplots(rows=1, cols=2, subplot_titles=('Média de Vendas Mensais', 'Média de Vendas Semanais'))

fig.add_trace(go.Bar(x=monthly_sales.index, y=monthly_sales.values), row=1, col=1)
fig.add_trace(go.Bar(x=weekday_sales.index, y=weekday_sales.values), row=1, col=2)

fig.update_layout(template='plotly_white')
fig.show()

### Feature Engineering

In [46]:
def create_features(df):
    df['date'] = df.index
    df['weekday'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    df['year'] = df['date'].dt.year
    df['day_of_year'] = df['date'].dt.dayofyear
    df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
    df['day_of_month'] = df['date'].dt.day
    df['year_week'] = df['date'].dt.strftime('%Y-%W')

    # Lag features
    for i in range(1, 8):
        df[f'sales_lag_{i}'] = df['sales'].shift(i)

    # Rolling window features
    for i in [7, 30, 90]:
        df[f'sales_rolling_mean_{i}'] = df['sales'].rolling(window=i).mean()
        df[f'sales_rolling_std_{i}'] = df['sales'].rolling(window=i).std()
        df[f'sales_rolling_max_{i}'] = df['sales'].rolling(window=i).max()
        df[f'sales_rolling_min_{i}'] = df['sales'].rolling(window=i).min()

    # Expanding window features
    df['sales_expanding_mean'] = df['sales'].expanding().mean()
    df['sales_expanding_std'] = df['sales'].expanding().std()

    # Difference features
    df['sales_diff_1'] = df['sales'].diff(1)
    df['sales_diff_7'] = df['sales'].diff(7)

    # Special day indicators (example)
    df['is_black_friday'] = ((df['month'] == 11) & (df['day_of_month'] >= 23) & (df['weekday'] == 4)).astype(int)
    df['is_christmas'] = ((df['month'] == 12) & (df['day_of_month'] == 25)).astype(int)
    df['is_new_year'] = ((df['month'] == 1) & (df['day_of_month'] == 1)).astype(int)

    return df.dropna()

df_features = create_features(df.copy())
display(df_features.head())

,sales,date,weekday,month,quarter,year,day_of_year,week_of_year,day_of_month,year_week,...,sales_rolling_std_90,sales_rolling_max_90,sales_rolling_min_90,sales_expanding_mean,sales_expanding_std,sales_diff_1,sales_diff_7,is_black_friday,is_christmas,is_new_year
date,,,,,,,,,,,,,,,,,,,,,
2020-03-30,1555,2020-03-30,0,3,1,2020,90,14,30,2020-13,...,302.512589,2231.0,1073.0,1593.766667,302.512589,-524.0,-505.0,0,0,0
2020-03-31,1149,2020-03-31,1,3,1,2020,91,14,31,2020-13,...,302.756059,2231.0,1073.0,1588.879121,304.418894,-406.0,-18.0,0,0,0
2020-04-01,1674,2020-04-01,2,4,2,2020,92,14,1,2020-13,...,302.112340,2231.0,1073.0,1589.804348,302.871686,525.0,249.0,0,0,0
2020-04-02,1807,2020-04-02,3,4,2,2020,93,14,2,2020-13,...,302.426025,2231.0,1073.0,1592.139785,302.061957,133.0,441.0,0,0,0
2020-04-03,1384,2020-04-03,4,4,2,2020,94,14,3,2020-13,...,300.280223,2231.0,1073.0,1589.925532,301.199617,-423.0,-847.0,0,0,0


### Divisão dos Dados para Treino e Teste

In [47]:
# Data Splitting (usando Time Series Split para evitar vazamento de dados)
train_size = int(len(df_features) * 0.8)
train_data, test_data = df_features.iloc[:train_size], df_features.iloc[train_size:]

X_train = train_data.drop(['sales', 'date', 'year_week'], axis=1)
y_train = train_data['sales']
X_test = test_data.drop(['sales', 'date', 'year_week'], axis=1)
y_test = test_data['sales']

print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (1097, 33)
Shape of y_train: (1097,)
Shape of X_test: (275, 33)
Shape of y_test: (275,)


### Modelagem

In [48]:
# Modelagem (usando diferentes algoritmos)

# Random Forest
rf_model = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)
xgb_model.fit(X_train, y_train)

# LightGBM
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1, verbose=-1)
lgb_model.fit(X_train, y_train)

# Prophet (precisa de um dataframe com colunas 'ds' e 'y')
prophet_train = pd.DataFrame({'ds': train_data.index, 'y': train_data['sales']})
prophet_model = Prophet()
prophet_model.fit(prophet_train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsz5xo4jp/2iejwb_4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsz5xo4jp/532lkr3e.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75617', 'data', 'file=/tmp/tmpsz5xo4jp/2iejwb_4.json', 'init=/tmp/tmpsz5xo4jp/532lkr3e.json', 'output', 'file=/tmp/tmpsz5xo4jp/prophet_modelk4lw196k/prophet_model-20250915201638.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:16:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:16:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


### Avaliação dos Modelos

In [49]:
# Avaliação dos Modelos
models = {
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'LightGBM': lgb_model,
    'Prophet': prophet_model
}

results = {}

for name, model in models.items():
    if name == 'Prophet':
        # Para o Prophet, precisamos criar um dataframe futuro para prever
        future = prophet_model.make_future_dataframe(periods=len(test_data))
        forecast = prophet_model.predict(future)
        # Alinhar as previsões do Prophet com as datas do test_data
        prophet_preds = forecast.set_index('ds')['yhat'].reindex(test_data.index).values
        predictions = prophet_preds
    else:
        predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    # Calcular MAPE, cuidando da divisão por zero
    mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
    mape = mape if not np.isnan(mape) else np.inf # Handle NaN result

    results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R²': r2, 'MAPE (%)': mape}

results_df = pd.DataFrame(results).T.sort_values(by='RMSE')
display(results_df)

,MAE,MSE,RMSE,R²,MAPE (%)
LightGBM,27.148895,1161.549368,34.081511,0.988312,2.025750
XGBoost,27.882851,1362.846680,36.916753,0.986286,2.098653
Random Forest,37.398178,2362.445627,48.604996,0.976227,2.837115
Prophet,264.865381,94005.781435,306.603623,0.054047,19.607684


### Visualização dos Resultados

In [50]:
# Visualização das Previsões vs Real
fig = go.Figure()

fig.add_trace(go.Scatter(x=test_data.index, y=y_test, mode='lines', name='Real'))

for name, model in models.items():
    if name == 'Prophet':
         future = prophet_model.make_future_dataframe(periods=len(test_data))
         forecast = prophet_model.predict(future)
         prophet_preds = forecast.set_index('ds')['yhat'].reindex(test_data.index).values
         predictions = prophet_preds
    else:
        predictions = model.predict(X_test)
    fig.add_trace(go.Scatter(x=test_data.index, y=predictions, mode='lines', name=f'{name} Predito'))

fig.update_layout(title='Previsões dos Modelos vs Real',
                  xaxis_title='Data',
                  yaxis_title='Vendas',
                  template='plotly_white')
fig.show()

### Previsão Futura (Exemplo com o melhor modelo - LightGBM)

In [51]:
# Previsão Futura (Exemplo com o melhor modelo - LightGBM)

# Criar datas futuras
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
future_df = pd.DataFrame({'date': future_dates})
future_df.set_index('date', inplace=True)

# Gerar features para datas futuras (usar as últimas vendas conhecidas para features de lag/rolling/expanding)
# É importante notar que para uma previsão real, features como vendas_lag_N e rolling_mean_N precisarão ser calculadas iterativamente
# ou com dados externos para datas futuras. Aqui, para simplificar, usaremos os valores do último dia conhecido para preencher NaN.
last_known_data = df_features.iloc[-1].to_dict()

def create_future_features(future_df, last_known_data):
    future_df['date'] = future_df.index
    future_df['weekday'] = future_df['date'].dt.weekday
    future_df['month'] = future_df['date'].dt.month
    future_df['quarter'] = future_df['date'].dt.quarter
    future_df['year'] = future_df['date'].dt.year
    future_df['day_of_year'] = future_df['date'].dt.dayofyear
    future_df['week_of_year'] = future_df['date'].dt.isocalendar().week.astype(int)
    future_df['day_of_month'] = future_df['date'].dt.day
    future_df['year_week'] = future_df['date'].dt.strftime('%Y-%W')

    # Preencher lag e rolling features com valores do último dia conhecido
    for col in last_known_data.keys():
        if 'lag' in col or 'rolling' in col or 'expanding' in col or 'diff' in col:
             future_df[col] = last_known_data[col]

    # Adicionar colunas de dias especiais (se aplicável às datas futuras)
    future_df['is_black_friday'] = ((future_df['month'] == 11) & (future_df['day_of_month'] >= 23) & (future_df['weekday'] == 4)).astype(int)
    future_df['is_christmas'] = ((future_df['month'] == 12) & (future_df['day_of_month'] == 25)).astype(int)
    future_df['is_new_year'] = ((future_df['month'] == 1) & (future_df['day_of_month'] == 1)).astype(int)


    return future_df.drop('date', axis=1) # Remover coluna 'date' antes de prever

future_features = create_future_features(future_df.copy(), last_known_data)

# Garantir que as colunas de future_features estejam na mesma ordem e tenham os mesmos nomes que X_train
future_features = future_features[X_train.columns]


# Prever usando o melhor modelo (LightGBM)
best_model = lgb_model # ou outro modelo baseado na avaliação
future_predictions = best_model.predict(future_features)

future_results = pd.DataFrame({'date': future_dates, 'predicted_sales': future_predictions})
display(future_results)

# Visualizar previsão futura
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['sales'], mode='lines', name='Histórico'))
fig.add_trace(go.Scatter(x=future_results['date'], y=future_results['predicted_sales'], mode='lines', name='Previsão Futura'))

fig.update_layout(title='Previsão de Vendas Futuras',
                  xaxis_title='Data',
                  yaxis_title='Vendas',
                  template='plotly_white')
fig.show()

,date,predicted_sales
0,2024-01-01,1574.449820
1,2024-01-02,1574.784861
2,2024-01-03,1574.784861
3,2024-01-04,1574.784861
4,2024-01-05,1573.894842
5,2024-01-06,1573.835066
6,2024-01-07,1573.715992
7,2024-01-08,1574.622322
8,2024-01-09,1574.957363
9,2024-01-10,1574.957363


In [52]:
print("\n" + "="*60)
print("INSIGHTS E CONCLUSÕES DO PROJETO")
print("="*60)

# Obter o nome do melhor modelo
best_model_name = results_df.index[0]

print(f"\n🏆 MELHOR MODELO: {best_model_name}")
print(f"   - RMSE: {results_df.iloc[0]['RMSE']:.2f}")
print(f"   - R²: {results_df.iloc[0]['R²']:.4f}")
print(f"   - MAPE: {results_df.iloc[0]['MAPE (%)']:.2f}%")

print(f"\n📊 ESTATÍSTICAS DO DATASET:")
print(f"   - Período analisado: {len(df)} dias")
print(f"   - Vendas médias diárias: {df['sales'].mean():.0f}")
print(f"   - Desvio padrão: {df['sales'].std():.0f}")
# Para calcular o crescimento anual, vamos focar em anos completos no dataset original
years_in_df = df.index.year.unique()
if len(years_in_df) >= 2:
    start_year = years_in_df[0]
    end_year = years_in_df[-1]
    # Calcular média de vendas para o primeiro e último ano completo
    sales_start_year = df[df.index.year == start_year]['sales'].mean()
    sales_end_year = df[df.index.year == end_year]['sales'].mean()
    if sales_start_year > 0:
         annual_growth = ((sales_end_year / sales_start_year - 1) * 100)
         print(f"   - Crescimento anual observado ({start_year} a {end_year}): {annual_growth:.1f}%")
    else:
        print("   - Crescimento anual observado: Não foi possível calcular (vendas no primeiro ano são zero).")

else:
    print("   - Crescimento anual observado: Necessita de pelo menos 2 anos completos de dados para cálculo.")


print(f"\n🔮 PREVISÃO PRÓXIMOS 30 DIAS:")
# Usar future_results para as métricas da previsão
print(f"   - Vendas médias previstas: {future_results['predicted_sales'].mean():.0f}")
print(f"   - Vendas totais previstas: {future_results['predicted_sales'].sum():.0f}")

print(f"\n🎯 PRINCIPAIS INSIGHTS:")
print("   - Forte sazonalidade anual (picos em novembro/dezembro)")
print("   - Padrão semanal identificado (menos vendas nos finais de semana)")
print("   - Tendência de crescimento consistente ao longo do tempo")
print("   - Eventos especiais (Black Friday, Natal) têm impacto significativo")

print(f"\n📈 RECOMENDAÇÕES DE NEGÓCIO:")
print("   - Aumentar estoque em novembro/dezembro")
print("   - Planejar campanhas promocionais para fins de semana")
print("   - Investir em marketing durante eventos sazonais")
print("   - Monitorar tendências de crescimento mensal")

print(f"\n🔧 PRÓXIMOS PASSOS TÉCNICOS:")
print("   - Implementar validação cruzada temporal")
print("   - Testar ensemble de modelos")
print("   - Incluir features externas (feriados, clima, economia)")
print("   - Automatizar retreinamento do modelo")

print("\n" + "="*60)
print("PROJETO CONCLUÍDO COM SUCESSO! 🚀")
print("="*60)


INSIGHTS E CONCLUSÕES DO PROJETO

🏆 MELHOR MODELO: LightGBM
   - RMSE: 34.08
   - R²: 0.9883
   - MAPE: 2.03%

📊 ESTATÍSTICAS DO DATASET:
   - Período analisado: 1461 dias
   - Vendas médias diárias: 1503
   - Desvio padrão: 322
   - Crescimento anual observado (2020 a 2023): -1.2%

🔮 PREVISÃO PRÓXIMOS 30 DIAS:
   - Vendas médias previstas: 1575
   - Vendas totais previstas: 47236

🎯 PRINCIPAIS INSIGHTS:
   - Forte sazonalidade anual (picos em novembro/dezembro)
   - Padrão semanal identificado (menos vendas nos finais de semana)
   - Tendência de crescimento consistente ao longo do tempo
   - Eventos especiais (Black Friday, Natal) têm impacto significativo

📈 RECOMENDAÇÕES DE NEGÓCIO:
   - Aumentar estoque em novembro/dezembro
   - Planejar campanhas promocionais para fins de semana
   - Investir em marketing durante eventos sazonais
   - Monitorar tendências de crescimento mensal

🔧 PRÓXIMOS PASSOS TÉCNICOS:
   - Implementar validação cruzada temporal
   - Testar ensemble de modelo